In [24]:
import os
import sys
sys.path.insert(0, "../src")
import tensorflow as tf

import gym
import baselines
from baselines import deepq

import normalized_env

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
import math
import numpy as np

def modify_env(env):

    # hack to give us control over the initial state
    import types

    def new_timelimt_step(self, action):
        assert self._episode_started_at is not None, "Cannot call env.step() before calling reset()"
        observation, reward, done, info = self.env.step(action)
        self._elapsed_steps += 1

        if self._past_limit():
            if self.metadata.get('semantics.autoreset'):
                _ = self.reset() # automatically reset the env
            done = True 

        return observation, reward, done, info

    def new_step(self, discrete_action):
        
            
        #disc_to_cont = {
        #    0: [-1.0],
        #    1: [-0.5],
        #    2: [0.0],
        #    3: [0.5],
        #    4: [1.0]
        #}
        
        disc_to_cont = {0: [-1.0],
         1: [-0.8],
         2: [-0.6],
         3: [-0.3999999999999999],
         4: [-0.19999999999999996],
         5: [0.0],
         6: [0.20000000000000018],
         7: [0.40000000000000013],
         8: [0.6000000000000001],
         9: [0.8],
         10: [1.0]}

        
        #n_actions = 30
        #disc_to_cont = {0: [-1.0],
        # 1: [-0.9310344827586207],
        # 2: [-0.8620689655172413],
        # 3: [-0.7931034482758621],
        # 4: [-0.7241379310344828],
        # 5: [-0.6551724137931034],
         #6: [-0.5862068965517242],
        # 7: [-0.5172413793103449],
        # 8: [-0.4482758620689655],
        # 9: [-0.3793103448275862],
        # 10: [-0.31034482758620685],
         #11: [-0.24137931034482762],
         #12: [-0.1724137931034483],
         #13: [-0.10344827586206895],
         #14: [-0.034482758620689724],
         #15: [0.034482758620689724],
         #16: [0.10344827586206895],
         #17: [0.17241379310344818],
         #18: [0.24137931034482762],
        # 19: [0.31034482758620685],
         #20: [0.3793103448275863],
         #21: [0.4482758620689655],
         #22: [0.5172413793103448],
        # 23: [0.5862068965517242],
        # 24: [0.6551724137931034],
        # #25: [0.7241379310344827],
         #26: [0.7931034482758621],
       #  27: [0.8620689655172413],
         #28: [0.9310344827586206],
         #29: [1.0]}
        
        action = disc_to_cont[int(discrete_action[0])]
        # hacky way to map continuous action to discrete value

        position = self.state[0]
        velocity = self.state[1]
        force = min(max(action[0], -1.0), 1.0)

        velocity += force*self.power -0.0025 * math.cos(3*position)
        if (velocity > self.max_speed): velocity = self.max_speed
        if (velocity < -self.max_speed): velocity = -self.max_speed
        position += velocity
        if (position > self.max_position): position = self.max_position
        if (position < self.min_position): position = self.min_position
        if (position==self.min_position and velocity<0): velocity = 0

        done = bool(position >= self.goal_position)

        reward = 0
        if done:
            reward = 100.0
        reward-= math.pow(action[0],2)*0.1

        self.state = np.array([position, velocity])

        return self.state, reward, done, {}
    # set max time step
    
    env._max_episode_steps = 999
    
    #env.reset = types.MethodType(new_timelimit_reset, env)
    env.env.step = types.MethodType(new_step, env.env)
    env = normalized_env.make_normalized_env(env)
    return env


In [31]:
gym.__version__

'0.10.5'

In [32]:
env_id = "MountainCarContinuous-v0"
env = gym.envs.make(env_id)
env = modify_env(env)
env._max_episode_steps

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
True action space: [-1.], [1.]
True state space: [-1.2  -0.07], [0.6  0.07]
Filtered action space: [-1.], [1.]
Filtered state space: [-1. -1.], [1. 1.]


999

In [35]:
def main():
    env_id = "MountainCarContinuous-v0"
    env = gym.envs.make(env_id)
    env = modify_env(env)
    model = deepq.models.mlp([200, 200], layer_norm=True)
    act = deepq.learn(
        env,
        n_action=11,
        q_func=model,
        model_path="output/DNN/",
        trial_i=1,
        lr=1e-3,
        max_timesteps=100000,
        buffer_size=100000,
        exploration_fraction=0.5,
        exploration_final_eps=0.05,
        print_freq=5,
        param_noise=True,
        target_network_update_freq=1000,
        gamma=0.99,
        prioritized_replay=True,
        prioritized_replay_alpha=0.6,
        prioritized_replay_beta0=0.9,
        prioritized_replay_beta_iters=100000,
        prioritized_replay_eps=1e-6
    )
    act.save("mountaincar_model.pkl")


In [36]:
with tf.Graph().as_default():
    main()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
True action space: [-1.], [1.]
True state space: [-1.2  -0.07], [0.6  0.07]
Filtered action space: [-1.], [1.]
Filtered state space: [-1. -1.], [1. 1.]


/home/srivatsan/.local/lib/python3.5/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/srivatsan/.local/lib/python3.5/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


--------------------------------------
| % time spent exploring  | 92       |
| episodes                | 5        |
| mean 100 episode reward | -64      |
| steps                   | 3995     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 82       |
| episodes                | 10       |
| mean 100 episode reward | -64      |
| steps                   | 8990     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 73       |
| episodes                | 15       |
| mean 100 episode reward | -66.5    |
| steps                   | 13985    |
--------------------------------------
--------------------------------------
| % time spent exploring  | 63       |
| episodes                | 20       |
| mean 100 episode reward | -65.8    |
| steps                   | 18980    |
--------------------------------------
--------------------------------------
| % time spent exploring 